In [17]:
from google.colab import files
files.upload()

Saving data_ploting.py to data_ploting.py


{'data_ploting.py': b'from PIL import Image\r\nimport numpy as np\r\nimport random\r\n\r\nimport matplotlib.pyplot as plt\r\nfrom matplotlib import cm\r\nimport seaborn as sns\r\n\r\nfrom mpl_toolkits.mplot3d import Axes3D\r\nimport matplotlib.patches as mpatches\r\nimport matplotlib.colors as mcolors\r\n\r\nimport os\r\nimport tempfile\r\nimport imageio\r\nimport shutil\r\n\r\n# set theme\r\nsns.set_context("poster", font_scale=0.8, rc={"lines.linewidth": 1.1})\r\n\r\ndef get_color():\r\n    color = random.choice(["mediumspringgreen",\r\n                           "cyan",\r\n                           "darkcyan",\r\n                           "mediumvioletred",\r\n                           "seagreen",\r\n                           "purple",\r\n                           "darkmagenta"])\r\n    return color\r\n\r\n\r\ndef tsne_plot_2d(filename, label, embeddings, words=[], a=1, color=get_color()):\r\n    sns.set_context("poster", font_scale=0.8, rc={"lines.linewidth": 1.1})\r\n    plt.

In [0]:
# Pubmed url
GETURL1 = "https://www.ncbi.nlm.nih.gov/pubmed/?term=Coronavirus+2019-nCoV"
NUM_PAGES1 = 1

# Biorxiv url
GETURL2 = "https://www.biorxiv.org/search/Coronavirus%252B2019-nCoV%20numresults%3A75%20sort%3Arelevance-rank?page="
NUM_PAGES2 = 1

MASK = "mask.png"
PROMO = "promo.jpg"
THEME = "Coronavirus 2019-nCoV"

In [4]:
!pip install fpdf aiohttp -qq

     |████████████████████████████████| 1.2MB 8.1MB/s 
     |████████████████████████████████| 153kB 53.4MB/s 
     |████████████████████████████████| 256kB 51.4MB/s 


In [5]:
%%time

import asyncio
import aiohttp
import socket

#import requests
from bs4 import BeautifulSoup


#GETURL = "https://www.ncbi.nlm.nih.gov/pubmed/?term=coronavirus+2019-nCoV"
POSTURL = "https://www.ncbi.nlm.nih.gov/pubmed/"


PMID = []
#NUM_PAGES = 1#46
ITEMS_PER_PAGE = 100


pubmed_MAP = {}
SEMA = asyncio.BoundedSemaphore(300)

####################### Get all PMID #########################

async def get_session_and_inputs(page, items_per_page):

    headers = {"User-Agent" : "Mozilla/5.0", "Connection": "close"}
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.get(GETURL1) as response:
            data = await response.text()
            soup = BeautifulSoup(data, "lxml")

            inputs = {i['name']: i.get('value', '') for i
                      in soup.select('form#EntrezForm input[name]')}
            inputs['EntrezSystem2.PEntrez.PubMed.Pubmed_ResultsPanel.Pubmed_DisplayBar.PageSize'] = ITEMS_PER_PAGE
            inputs['EntrezSystem2.PEntrez.PubMed.Pubmed_ResultsPanel.Pubmed_DisplayBar.PrevPageSize'] = ITEMS_PER_PAGE
            inputs['EntrezSystem2.PEntrez.DbConnector.Cmd'] = 'PageChanged'

            inputs['EntrezSystem2.PEntrez.PubMed.Pubmed_ResultsPanel.Pubmed_Pager.CurrPage'] = page
            inputs['EntrezSystem2.PEntrez.PubMed.Pubmed_ResultsPanel.Pubmed_Pager.cPage'] = page

        async with session.post(POSTURL, data=inputs) as response2:
            data = await response2.text()
            soup = BeautifulSoup(data, "lxml")
            PMID.append([pmid.text for pmid in soup('dd')])


async def get_all_pmid():
    tasks = []
    for page in range(NUM_PAGES1):
        task = asyncio.ensure_future(get_session_and_inputs(page=page,
                                                            items_per_page=NUM_PAGES1))
        tasks.append(task)

    await asyncio.gather(*tasks)

############# Get all data in MAP from each uniq PMID articles ###################

async def parse_text_from_paper(pmid):
    '''
    From link to science paper (pmid) - get html,
    then get document's body (data) from one page.
    '''
    
    conn = aiohttp.TCPConnector(family=socket.AF_INET)
    headers={"User-Agent" : "Mozilla/5.0",
             "Connection": "close"}
    
    async with aiohttp.ClientSession(headers=headers, connector=conn) as session:
        async with SEMA, session.get(f"https://www.ncbi.nlm.nih.gov/pubmed/{pmid}") as response:
            data = await response.text()
            soup = BeautifulSoup(data, "lxml")

            title = [text.find('h1').text for text in soup.find_all("div", class_="rprt_all")]
            data = [text.find('p').text for text in soup.find_all("div", class_="abstr")]
            pubmed_MAP[POSTURL+pmid] = data, title[0]

async def get_all_data():

    tasks = []
    pmid_items = [pmid_i for lst in PMID for pmid_i in lst]

    for pmid in pmid_items:

        task = asyncio.ensure_future(parse_text_from_paper(pmid))
        tasks.append(task)

    await asyncio.gather(*tasks)


loop = asyncio.get_event_loop()
loop.run_until_complete(get_all_pmid())
loop = asyncio.get_event_loop()
loop.run_until_complete(get_all_data())
    
print(f"Done!\nMap length: {len(pubmed_MAP)}")

Done!
Map length: 72
CPU times: user 3.32 s, sys: 91.5 ms, total: 3.41 s
Wall time: 4.85 s


In [6]:
%%time

import asyncio
import aiohttp

import requests
from bs4 import BeautifulSoup

#GETURL2 = "https://www.biorxiv.org/search/coronavirus%252B2019-nCoV?page="
MAIN_URL = "https://www.biorxiv.org"
PAPER_LINKS = []
#NUM_PAGES2 = 12
biorxiv_MAP = {}

async def fetch_links_from_page(page, session, geturl=GETURL2):
    """
    corutine parse links on paper from page
    and added there into list.
    """

    async with session.get(f"{geturl}{page}") as response:
        data = await response.text()
        soup = BeautifulSoup(data, "lxml")

        PAPER_LINKS.append([MAIN_URL + i.get("href")for i in 
                         soup.find_all(name='a',class_='highwire-cite-linked-title')])



async def get_paper_links(url):
    tasks = []
    async with aiohttp.ClientSession() as session:
        for page in range(NUM_PAGES2):
            task = asyncio.ensure_future(fetch_links_from_page(page, session, geturl=url))
            tasks.append(task)
        
        await asyncio.gather(*tasks)


async def fetch_content(url, session):
    async with session.get(url) as response:
        data = await response.text()
        soup = BeautifulSoup(data, "lxml")
        try:
            title = soup.find("h1", {"id": "page-title"}).text
            biorxiv_MAP[url] = [p.text for p in 
                            soup.find("div", {"id": "abstract-1"}).find_all('p')], title
        except:
            print(f"\n Problem with this link {url}")           
            biorxiv_MAP[url] = None




async def get_all_data():
    tasks = []

    async with aiohttp.ClientSession() as session:
        URLS = [link for lst in PAPER_LINKS for link in lst]
        for url in URLS:
            task = asyncio.ensure_future(fetch_content(url, session))
            tasks.append(task)
        await asyncio.gather(*tasks)




loop = asyncio.get_event_loop()
loop.run_until_complete(get_paper_links(GETURL2))

loop.run_until_complete(get_all_data())

CPU times: user 1.52 s, sys: 29.3 ms, total: 1.55 s
Wall time: 6.33 s


In [7]:
def remove_none(data):
    print(len(data))
    filtered = {k: v for k, v in data.items() if v is not None}
    data.clear()
    data.update(filtered)
    print(len(data))
    return data

pubmed_MAP = remove_none(pubmed_MAP)
biorxiv_MAP = remove_none(biorxiv_MAP)

72
72
42
42


In [0]:
from PIL import Image
import numpy as np

def get_mask(img_path):
    img = Image.open(img_path).convert("L")
    return np.array(img)

MASK = get_mask(MASK)

In [11]:
pubmed_MAP

{'https://www.ncbi.nlm.nih.gov/pubmed/31944312': (["Respiratory tract viral infection caused by viruses or bacteria is one of the most common diseases in human worldwide, while those caused by emerging viruses, such as the novel coronavirus, 2019-nCoV that caused the pneumonia outbreak in Wuhan, China most recently, have posed great threats to global public health. Identification of the causative viral pathogens of respiratory tract viral infections is important to select an appropriate treatment, save people's lives, stop the epidemics, and avoid unnecessary use of antibiotics. Conventional diagnostic tests, such as the assays for rapid detection of antiviral antibodies or viral antigens, are widely used in many clinical laboratories. With the development of modern technologies, new diagnostic strategies, including multiplex nucleic acid amplification and microarray-based assays, are emerging. This review summarizes currently available and novel emerging diagnostic methods for the det

In [19]:
from main import main
import nltk
import re
nltk.download('stopwords') 
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
%%time
main(p_data=pubmed_MAP,
     b_data=biorxiv_MAP, 
     p_gif_name="3d_pubmded.gif",
     b_gif_name="3d_bioxriv.gif",
     p_wc_name="pubmed_wordcloud",
     b_wc_name_mask="bioxriv_wordcloud_mask",
     b_wc_name="bioxriv_wordcloud",
     p_wc_name_mask="pubmed_wordcloud_mask",
     p_pdf_name="pubmed",
     b_pdf_name="bioxriv",
     p_site="PubMed",
     b_site="bioRxiv", 
     b_2d_name1="biorxiv_tsne_2d_dots.png",
     b_2d_name2="biorxiv_tsne_2d_words.png",
     p_2d_name1="pubmed_tsne_2d_dots.png",
     p_2d_name2="pubmed_tsne_2d_words.png", 
     b_sim_name="bioxriv_similar_words.png", 
     p_sim_name="pubmed_similar_words.png",
     theme=THEME,
     mask=MASK, 
     img_promo=PROMO)

Num articles: 72
Num senteces: 230
Num articles: 42
Num senteces: 259


/usr/local/lib/python3.6/dist-packages/matplotlib/collections.py:857: RuntimeWarning: invalid value encountered in sqrt
  scale = np.sqrt(self._sizes) * dpi / 72.0 * self._factor
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Amount of empty data: 20
Amount of empty data: 0
Done!
CPU times: user 7min 55s, sys: 6.45 s, total: 8min 1s
Wall time: 6min 25s


In [21]:
from IPython.display import FileLink, FileLinks
FileLinks('/content/')

/content/
  data_ploting.py
  3d_bioxriv.gif
  mask.png
  w2v_tsne.py
  words_cloud.py
  biorxiv_tsne_2d_words.png.png
  bioxriv_wordcloud_mask.png
  pubmed_tsne_2d_dots.png.png
  pubmed_wordcloud_mask.png
  bioxriv_similar_words.png.png
  bioxriv.pdf
  3d_pubmded.gif
  biorxiv_tsne_2d_dots.png.png
  bioxriv_wordcloud.png
  get_pdf.py
  promo.jpg
  pubmed_wordcloud.png
  main.py
  pubmed_similar_words.png.png
  pubmed_tsne_2d_words.png.png
  pubmed.pdf
/content/.config/
  .last_survey_prompt.yaml
  active_config
  gce
  config_sentinel
  .last_update_check.json
  .metricsUUID
/content/.config/configurations/
  config_default
/content/.config/logs/2020.02.05/
  18.37.01.637056.log
  18.36.15.774867.log
  18.36.48.819796.log
  18.36.32.968039.log
  18.37.02.083241.log
  18.36.45.012839.log
/content/__pycache__/
  get_pdf.cpython-36.pyc
  main.cpython-36.pyc
  words_cloud.cpython-36.pyc
  data_ploting.cpython-36.pyc
  w2v_tsne.cpython-36.pyc
/content/sample_data/
  anscombe.json
  README.md
  california_housing_train.csv
  mnist_train_small.csv
  california_housing_test.csv
  mnist_test.csv